# Глубокие Нейронные сетти

## Библиотеки

In [1]:
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import torch

from torchvision import datasets, transforms


In [2]:
import warnings
warnings.filterwarnings("ignore")